In [24]:
import os
import requests
import json
import pandas as pd
from mysql import connector
from dotenv import load_dotenv
import numpy as np
from datetime import datetime

In [25]:
coins = ['bitcoin', 'solana']

load_dotenv()

url_base = "https://api.coingecko.com/api/v3/coins/"
days = 5
params = {
    "vs_currency": "usd",
    "days": days
}

print(params)

{'vs_currency': 'usd', 'days': 5}


# EXTRACT

In [26]:
payloads = []


for idx, coin in enumerate(coins):
    url = f"{url_base}{coin}/market_chart"
    response = requests.get(url, params=params)
    payloads.append(response.json())
    
formatted_response = json.dumps(payloads, indent=4)

# TRANSFORM

In [27]:
market_history_df1 = pd.DataFrame(payloads)
market_history_df1

,prices,market_caps,total_volumes
0,"[[1758553353319, 113030.16943960388], [1758556...","[[1758553353319, 2251830994047.513], [17585569...","[[1758553353319, 58937369008.865074], [1758556..."
1,"[[1758553352602, 221.61247002124892], [1758556...","[[1758553352602, 120372088410.52043], [1758556...","[[1758553352602, 10350190311.866674], [1758556..."


In [28]:
def prepare_market_history_records(coin_id, market_chart_data, sample_size=(3 * days)):
    rows = []
    total_records = len(market_chart_data['prices'])
    indices = np.linspace(0, total_records - 1, sample_size)
    indices = np.floor(indices).astype(int)

    print(indices)

    for i in indices:
        timestamp_ms = market_chart_data['prices'][i][0]
        timestamp = datetime.fromtimestamp(timestamp_ms / 1000)

        price = round(market_chart_data['prices'][i][1], 5)
        market_cap = round(market_chart_data['market_caps'][i][1], 5)
        volume = round(market_chart_data['total_volumes'][i][1], 5)

        row = {
            'coin_id': coin_id,
            'timestamp': timestamp,
            'price': price,
            'market_cap': market_cap,
            'volume': volume
        }
        rows.append(row)

    return rows

fixed_payload = []
for idx, coin in enumerate(coins):
    fixed_payload.extend(prepare_market_history_records(coin, payloads[idx]))

market_history_df = pd.DataFrame(fixed_payload)
market_history_df


[  0   8  17  25  34  42  51  60  68  77  85  94 102 111 120]
[  0   8  17  25  34  42  51  60  68  77  85  94 102 111 120]


,coin_id,timestamp,price,market_cap,volume
0,bitcoin,2025-09-22 11:02:33.319,113030.16944,2.251831e+12,5.893737e+10
1,bitcoin,2025-09-22 19:02:41.272,112696.45088,2.245072e+12,6.884217e+10
2,bitcoin,2025-09-23 04:02:58.851,113095.48038,2.253196e+12,5.549140e+10
3,bitcoin,2025-09-23 12:01:38.790,112925.49388,2.249578e+12,4.177938e+10
4,bitcoin,2025-09-23 21:02:05.972,112234.63461,2.237332e+12,4.642758e+10
5,bitcoin,2025-09-24 05:02:43.466,112561.71502,2.242958e+12,4.742019e+10
6,bitcoin,2025-09-24 14:02:52.644,113833.03808,2.266963e+12,4.940124e+10
7,bitcoin,2025-09-24 23:02:27.344,112874.55771,2.249255e+12,4.773780e+10
8,bitcoin,2025-09-25 07:02:37.593,111675.26781,2.225289e+12,5.142772e+10
9,bitcoin,2025-09-25 16:01:55.637,109479.26783,2.181532e+12,7.172643e+10


# LOAD

In [17]:
MYSQL_USER = os.getenv("MYSQL_USER")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")
MYSQL_HOST = os.getenv("MYSQL_HOST")
MYSQL_PORT = os.getenv("MYSQL_PORT")
MYSQL_DB = os.getenv("MYSQL_DATABASE")

In [18]:
db_conn = connector.connect(
    host=MYSQL_HOST,
    user=MYSQL_USER,
    password=MYSQL_PASSWORD,
    port=MYSQL_PORT,
    database=MYSQL_DB,
    connection_timeout=10,
    autocommit=False,
    raise_on_warnings=True
)   

db_cur = db_conn.cursor()
print(f"[SUCCESS] Connected to MySQL db {MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB} as user {MYSQL_USER}")

[SUCCESS] Connected to MySQL db localhost:3306/cryptodb as user root


In [19]:
sql_table = "market_history"
db_cur.execute(f"SHOW TABLES LIKE '{sql_table}'")

if db_cur.fetchone() is None:
    raise SystemExit(f"[ERROR] Table '{sql_table}' does not exist in database '{MYSQL_DB}'")
else:
    print(f"[SUCCESS] Table '{sql_table}' exists in database '{MYSQL_DB}'")

[SUCCESS] Table 'market_history' exists in database 'cryptodb'


In [ ]:
INSERT_SQL = f"""
INSERT INTO {sql_table}] (
    coin_id, timestamp, price, market_cap, volume
) VALUES (
    %s, %s, %s, %s, %s
)
"""

In [21]:
market_history_list = market_history_df.values.tolist()

In [22]:
try:
    db_cur.executemany(INSERT_SQL, market_history_list)
    db_conn.commit()
    print(f"[SUCCESS] Inserted {db_cur.rowcount} records into table '{sql_table}'")
except connector.Error as err:
    db_conn.rollback()
    print(f"[ERROR] Failed to insert records into table '{sql_table}': {err}")
finally:
    db_cur.close()
    db_conn.close()
    print("[INFO] MySQL connection closed")

[SUCCESS] Inserted 540 records into table 'market_history'
[INFO] MySQL connection closed
